In [9]:
import numpy as np
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical 

def get_encoder(input_shape=(512,)):
    model = Sequential()
    model.add(Dense(512, activation='relu',
                    input_shape=input_shape))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(128, activation="sigmoid"))
    return model

def get_task(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(34, activation="softmax"))
    return model

def get_discriminator(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

save_check_pt = './checkpoints_DANN'
# Sample to load model
#Dummy inputs to load model
xs=np.zeros((8446, 512))
ys=np.zeros((8446, 34))
xt=np.zeros((762, 512))
yt=np.zeros((762, 34))
weather_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
weather_model.fit(xs, ys,xt,yt, epochs=0,verbose=1,batch_size = 32) # NO fit happen, just dummy step to load model
weather_model.load_weights(save_check_pt+ '/weather')

#sample input and predict
sample_in = np.zeros(512) #flatten 512 pixels from 32 * 16 orginal image
predction = weather_model.predict(sample_in.reshape([-1,512]))
print(predction)# prediction is one-hot encoding
print(np.argmax(predction)) # actual predict outcome

[[1.75311667e-04 2.20407285e-02 1.17826668e-04 3.17635536e-02
  8.98859464e-03 2.70385877e-04 1.27576008e-01 7.73248635e-03
  1.39435015e-05 2.15896216e-04 1.02603761e-03 1.54712819e-04
  6.32272349e-05 4.29541687e-05 5.26674849e-04 7.20039825e-04
  1.11301949e-06 2.88783312e-01 4.42625098e-02 1.18265720e-03
  2.82189772e-02 8.90552849e-02 1.09257447e-02 1.14442408e-03
  1.70786176e-02 1.10733730e-03 8.42365203e-04 6.80524856e-04
  4.84570861e-02 1.05735734e-01 3.37281053e-05 7.16076493e-02
  8.75335932e-02 1.92103663e-03]]
17
